OData API

https://oda.ft.dk/Home/OdataQuery

https://www.odata.org/

https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx

https://www.odata.org/documentation/odata-version-3-0/url-conventions/

https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/

Source root Link: "https://oda.ft.dk/api/"

ftp://oda.ft.dk/ODAXML/Referat/samling/


In [7]:
import requests
import urllib.request
import os
import io
import json

import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import lxml

In [2]:
#pip install bs4 lxml

In [9]:
path = os.getcwd()

In [11]:
path = path+'/Referater - 20091_M3'

In [12]:
path

'/Users/frederikkromannhansen/Documents/GitHub/politiker/Referater - 20091_M3'

In [13]:
for _, _, files in os.walk(path):
    for file in files:
        print(file)
        text = io.open(file,'rt',encoding = 'utf-8').read()
##            marvel_words += nltk.word_tokenize(marvel_text)

20091_M13_helemoedet.xml


FileNotFoundError: [Errno 2] No such file or directory: '20091_M13_helemoedet.xml'

In [45]:
referater = []

for _, _, files in os.walk(path):
    for file in files:
        with open(path+file, "r") as referat:
        # Read each line in the file, readlines() returns a list of lines
            content = referat.read()
        # Combine the lines in the list into a string
        #content = "".join(content)
            referater += bs(content, "lxml")

In [51]:
referater[1]

<html><body><p>﻿<?xml version="1.0" encoding="utf-8"?><dokument edixistatus=""><metameeting><parliamentarysession>20091</parliamentarysession><parliamentarygroup>Folketinget</parliamentarygroup><meetingnumber>103</meetingnumber><dateofsitting>2010-06-03T10:00:00</dateofsitting><location>Folketingssalen</location><edixidoclocation>W:\20091\M103\</edixidoclocation><audiofilefolder>X:\DAudio\FT2009\103\SALEN\</audiofilefolder></metameeting><titelgruppe><titel><linea><char>103. møde</char></linea></titel><undertitel><linea><char>Torsdag den 3. juni 2010 kl. 10.00</char></linea></undertitel></titelgruppe><dagsordenplan><rubrica>
<linea>
<char>Dagsorden</char>
</linea>
</rubrica>
<punkttekst>
<exitus>
<linea>
<char>1) 3. behandling af lovforslag nr. L 193:</char>
</linea>
</exitus>
<exitus>
<linea>
<char>Forslag til lov om udlodning af overskud fra lotteri og heste- og hundevæddemål.</char>
</linea>
</exitus>
<exitus>
<linea>
<char>Af kulturministeren (Per Stig Møller).</char>
</linea>
</exi

In [ ]:
print(bs_content.find_all('taler'))

In [ ]:
bs_content.find_all('taler')

In [ ]:
url = 'ftp://oda.ft.dk/ODAXML/Referat/samling/20091/'
response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8')

In [ ]:
text

In [ ]:
response = urllib.request.urlopen(url)

In [ ]:
data = response.read()

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
print(soup.decode())

In [ ]:
root = "https://oda.ft.dk/api/"
ressource = "M%C3%B8de?"
query = "$top=1&$expand=Dagsordenspunkt"

In [ ]:
data

In [ ]:
stopwords = open('stopord.txt').read().splitlines() #Link: https://gist.github.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b

In [ ]:
# convert the text to a tf-idf weighted term-document matrix

# Hyperparameters
corpus = 2000000 #Der er ca. ? forskellige ord, efter stopord
l1_ratio = 0
max_iter = 50000
no_components = 30

min_df = 2
max_df = 0.6


vectorizer = TfidfVectorizer(max_features=corpus, min_df=min_df, max_df=max_df, stop_words=stopwords)

X = vectorizer.fit_transform(data)
 
idx_to_word = np.array(vectorizer.get_feature_names())


# apply NMF
nmf = NMF(n_components=no_components, solver="mu", l1_ratio = l1_ratio, max_iter = max_iter, random_state=1)

W = nmf.fit_transform(X)

H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

In [ ]:
# Write to a .csv file
import csv
with open('topics_and_weights.csv', 'w', newline='', encoding='utf-16') as csvfile:
    topicwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
# Create the header
    topicwriter.writerow(['Topic','Word','Weight'])

In [ ]:
# print the topics for tableau
 
for i, topic in enumerate(H):
 
    max_weights = topic.argsort()[-10:]
    words = [str(x) for x in idx_to_word[topic.argsort()[-10:]]]
     
    for count, item in enumerate(max_weights):
        
        with open('topics_and_weights.csv', 'a', newline='', encoding='utf-16') as csvfile:
            topicwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
            topicwriter.writerow(["Topic"+str(i+1) ,words[count],H[i,item]])

In [ ]:
#stopwords_edit = open('stopord.txt','a')
#stopwords_edit.write('\nenhedslisten')
#stopwords_edit.close()